<a href="https://colab.research.google.com/github/ashikita/pdf_adjustment/blob/main/pdf_encrypt_copy_restrict.py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 作業ディレクトリ内のファイルを削除

!rm -f ./*

In [ ]:
# outputディレクトリ内のファイルを削除

!rm -f output/*

In [ ]:

# -*- coding: utf-8 -*-
"""
PDFの本文コピーを制限する

"""

# === 設定ブロック ===
# 目的：本文コピー（抽出）を禁止しつつ、閲覧は無制限（パスワード不要）、印刷は許可。
USER_PASSWORD = ""                               # ← 開く時のパスワード不要
OWNER_PASSWORD = "set_a_strong_password_here"    # ← 必須：保護解除用。強固な値に変更してください。
ALLOW_ACCESSIBILITY = True                       # スクリーンリーダー向け抽出を許可するなら True
ALLOW_PRINT_HIGHRES = True                       # 高解像度印刷を許可する（低解像度のみ許可したい場合は False にして下の lowres を True）
ALLOW_PRINT_LOWRES = False                       # 低解像度印刷のみ許可したい場合は True

# ---- 依存パッケージ（必要なら pip で自動インストール）----
import sys
import subprocess

def _pip_install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", pkg])

try:
    import pikepdf
    from pikepdf import Pdf, Permissions, Encryption
    _LIB = "pikepdf"
except ImportError:
    _pip_install("pikepdf")
    import pikepdf
    from pikepdf import Pdf, Permissions, Encryption
    _LIB = "pikepdf"

print(f"[INFO] Using {_LIB} {pikepdf.__version__}")


# === ここから実装 ===
import os
import time
import glob

def build_permissions():
    """
    印刷は許可、本文コピー（抽出）は禁止。必要に応じてアクセシビリティ抽出は許可。
    pikepdf の Permissions 引数例：
      - extract=False         → 内容のコピー/抽出禁止
      - print_highres=True    → 高解像度印刷許可（print_lowres=True でも可）
      - accessibility=True    → スクリーンリーダー向け抽出の許可
    """
    kwargs = {
        "extract": False,                           # ★本文コピー（抽出）を禁止
        "accessibility": ALLOW_ACCESSIBILITY,       # スクリーンリーダー向け抽出
    }
    # 印刷の許可（高解像度 / 低解像度のいずれかを許可）
    if ALLOW_PRINT_HIGHRES:
        kwargs["print_highres"] = True
    if ALLOW_PRINT_LOWRES:
        kwargs["print_lowres"] = True

    # どちらも False の場合は印刷不可になる点に注意
    return Permissions(**kwargs)

def protect_pdf(in_path, out_path):
    # 既存の暗号化状態に関係なく、保存時に新しい権限で暗号化する
    with Pdf.open(in_path) as pdf:
        perms = build_permissions()
        # Encryption の user は空文字（閲覧無制限）、owner は保護解除用に必須
        # R=6 は AES-256（Acrobat X 以降）。古いビューア互換が必要なら R を省略してください。
        enc = Encryption(
            user=USER_PASSWORD,
            owner=OWNER_PASSWORD,
            allow=perms,
            R=6
        )
        pdf.save(out_path, encryption=enc)

def main():
    os.makedirs("output", exist_ok=True)

    # カレント直下の PDF を対象（output 配下は対象外）
    candidates = []
    for pattern in ("*.pdf", "*.PDF", "*.Pdf", "*.pDf", "*.pdF"):
        candidates.extend(glob.glob(pattern))
    candidates = sorted(set(candidates))

    if not candidates:
        print("INFO: 対象となる PDF が見つかりませんでした。")
        return

    for pdf_path in candidates:
        start = time.perf_counter()
        status = "SUCCESS"
        err_msg = ""
        try:
            out_path = os.path.join("output", os.path.basename(pdf_path))
            protect_pdf(pdf_path, out_path)
        except Exception as e:
            status = "ERROR"
            err_msg = str(e)
        elapsed = time.perf_counter() - start

        # 要求仕様：ファイル名、成功・エラーのステータス、かかった時間を標準出力
        if status == "SUCCESS":
            print(f"{os.path.basename(pdf_path)},SUCCESS,{elapsed:.3f}s")
        else:
            print(f"{os.path.basename(pdf_path)},ERROR,{elapsed:.3f}s,{err_msg}")

if __name__ == "__main__":
    main()
